# Embedding 

The aim of this notebook is to test the embedding model and see if the results look logical for the naked eye. 
What is more, Bokeh and pyvis will be tested for generating the network graphs.

## Imports

In [3]:
# add root path to system path
import sys
sys.path.append('../')

In [4]:
import itertools
import pandas as pd
import json
import os
from tqdm import tqdm
from scripts.helpers import get_embeddings_from_json

In [ ]:
root_path = os.path.dirname(os.path.abspath(""))

## Data

Note that data preprocessing has been done in [preprocess.ipynb](preprocess.ipynb). The notebook will clean, merge and save the data to `data/processed`.

In [5]:
movie_df = pd.read_csv('../data/processed/preprocessed.csv', index_col=0)
print(movie_df.shape)
movie_df.head(5)

(22394, 10)


,name,release_year,rating,num_votes,wikipedia_id,plot,freebase_id,languages,countries,genres
0,Miss Jerry,1894.0,5.3,207,10109752,After finding out that her father is suffering...,/m/02q23xk,['Silent film'],['United States of America'],"['Short Film', 'Silent film', 'Indie', 'Black-..."
1,The Corbett-Fitzsimmons Fight,1897.0,5.3,484,28703057,The film no longer exists in its entirety; how...,/m/0czdh_n,[],[],['Sports']
2,The Story of the Kelly Gang,1906.0,6.0,855,142995,The Story of the Kelly Gangs tone is of sorrow...,/m/0120y4,['English Language'],['Australia'],"['Crime Fiction', 'Silent film', 'Biography', ..."
3,Robbery Under Arms,1907.0,4.3,25,32986669,Key scenes of the film included the branding o...,/m/04p7yxx,['Silent film'],['Australia'],"['Silent film', 'Drama']"
4,"Captain Midnight, the Bush King",1911.0,5.4,18,32987200,Edgar Dalimore is the son of wealthy station o...,/m/0h569x9,['Silent film'],[],['Silent film']


## Embeddings

Since we're already embedded the movies using Google Colab and downloaded them (see Readme), we can simply read the json.

In [6]:
DECADE = 2010
embeddings = get_embeddings_from_json(DECADE)

### Modelling

### Similarity Matrix

Once we have vectorized the plots, we can calculate the similarities between the plots with dot product.

In [7]:
# generate all possible combinations for dot product 
movie_sample = list(embeddings.keys())
combinations = list(itertools.combinations(movie_sample, 2))

In [8]:
similarity_dict = {}

for movie_1, movie_2 in tqdm(combinations):
    similarity = embeddings[movie_1] @ embeddings[movie_2]
    similarity_dict['-'.join([str(movie_1), str(movie_2)])] = similarity

100%|██████████| 200/200 [00:00<00:00, 15160.50it/s]


,27852285,9873472,25178358,20991625,28245273,12580152,13783850,619728,982480,26252728,...,28296022,35495683,34026052,36292850,31664578,35203478,35457037,32038154,33599927,29702115
27852285,NaN,0.722633,0.714975,0.7032,0.74843,0.704317,0.709198,0.71413,0.713284,0.716915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9873472,0.722633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25178358,0.714975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20991625,0.703200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28245273,0.748430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
with open(os.path.join(root_path, 'data', 'embeddings', 'similarities', f'similarities_{DECADE}s.json'), "w") as outfile: 
    json.dump(similarity_dict, outfile)